In [1]:
from keras import backend as K
import tensorflow as tf

def weighted_binary_crossentropy(y_true, y_pred):
    x = 0.10 #Approximate percentage of positives in each of the 17 zones
    POS_ADJ = 0.5/x
    NEG_ADJ = 0.5/(1-x)
    n_values = y_true.shape[0]
    elems = (tf.unstack(y_true,axis=0)) 
    adj = tf.map_fn(lambda x:tuple([tf.cond(tf.equal(x[i],1.),lambda:POS_ADJ,lambda: NEG_ADJ) for i in range(n_values)]),
                    elems, 
                    dtype = tuple([tf.float32 for i in range(n_values)]) )
    adj = tf.stack(adj,axis=0)
    print("Before averaging")
    print(tf.multiply(adj,K.binary_crossentropy(y_true,y_pred)).eval())
    return K.mean(tf.multiply(adj,K.binary_crossentropy(y_true,y_pred)),axis=-1)

Using TensorFlow backend.


In [3]:
import numpy as np
sess = tf.InteractiveSession()
a= np.array([[1,0,1,1],[1,1,1,1],[0,0,0,0]])
print(a.shape)
b= np.array([[0.5,0.5,0.5,0.5],[0.01,0.1,0.3,0.9],[0.5,0.7,0.5,0.2]])

print(b.shape)
z = weighted_binary_crossentropy(tf.convert_to_tensor(a,dtype=tf.float32),tf.convert_to_tensor(b,dtype=tf.float32))
print("Final result")
print(z.eval())

(3, 4)
(3, 4)
Before averaging
[[  3.46573591   0.3850818    3.46573591   3.46573591]
 [ 23.0258522   11.5129261    6.01986408   0.52680266]
 [  0.3850818    0.66887385   0.3850818    0.12396865]]
Final result
[  2.69557238  10.27136135   0.39075151]


In [43]:
from keras import backend as K
import tensorflow as tf
import numpy as np

sess = tf.InteractiveSession()
def ToRGB(x):
    max_v = tf.reduce_max(x)
    min_v = tf.reduce_min(x)
    max_rgb = tf.constant(255,dtype=x.dtype)
    x = tf.floordiv(tf.multiply(tf.subtract(x,min_v),max_rgb),tf.subtract(max_v,min_v))
    return x
def ToNewShape(x):
    ndim = len(x.shape)
    if ndim == 5:
        return tf.reverse(tf.transpose(x,[0,1,3,2,4]),[-3])
    elif ndim == 4:
        return tf.reverse(tf.transpose(x,[0,2,1,3]),[-3])
    else:
        raise ValueError("Unexpected number of dims!")

In [44]:
x = np.array([[13.5,100.45,45.23],[3434.43,445,3243],[3434,232,545]])
x = tf.convert_to_tensor(x,dtype=tf.int32)
y=ToRGB(x)
print(y.eval())

[[  0   6   2]
 [255  32 240]
 [255  16  39]]


In [47]:
x = np.reshape(np.arange(0,200),newshape=(2,2,5,10,1))
print(x[0,0,:,:,0])
x = tf.convert_to_tensor(x,tf.int32)
y = ToNewShape(x)
print(y.eval()[0,0,:,:,0])

[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [20 21 22 23 24 25 26 27 28 29]
 [30 31 32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47 48 49]]
[[ 9 19 29 39 49]
 [ 8 18 28 38 48]
 [ 7 17 27 37 47]
 [ 6 16 26 36 46]
 [ 5 15 25 35 45]
 [ 4 14 24 34 44]
 [ 3 13 23 33 43]
 [ 2 12 22 32 42]
 [ 1 11 21 31 41]
 [ 0 10 20 30 40]]


In [33]:
from keras import backend as K
import tensorflow as tf
import numpy as np

sess = tf.InteractiveSession()

x = np.array([[[[1],[2],[3]],[[4],[5],[6]],[[7],[8],[9]]],[[[10],[11],[12]],[[13],[14],[15]],[[16],[17],[18]]]])
print(x[0])
print(x.shape)
x = tf.convert_to_tensor(x,tf.int32)
z = [[1 for d in x.get_shape()[:-1]],[3]]
z = [val for sublist in z for val in sublist]
print(z)
def toGreyScale(x):
    shape = x.get_shape()
    #assume channel_last
    mult = [[1 for d in shape[:-1]],[3]]
    mult = [val for sublist in mult for val in sublist]
    return tf.tile(x,mult)
y = toGreyScale(x)
print(y.eval()[0])
print(y.shape)

[[[1]
  [2]
  [3]]

 [[4]
  [5]
  [6]]

 [[7]
  [8]
  [9]]]
(2, 3, 3, 1)
[1, 1, 1, 3]
[[[1 1 1]
  [2 2 2]
  [3 3 3]]

 [[4 4 4]
  [5 5 5]
  [6 6 6]]

 [[7 7 7]
  [8 8 8]
  [9 9 9]]]
(2, 3, 3, 3)
